In [1]:
from beyondllm import source, llms, retrieve, generator
from beyondllm.embeddings import FineTuneEmbeddings

In [5]:
from getpass import getpass
import os

google_api_key = getpass('Enter Your Google API Key')
os.environ['GOOGLE_API_KEY'] = google_api_key

Enter Your Google API Key ········


In [3]:
path = "DS-Alvin Rachmat 28 Aug 24 v4.32.pdf"
data = source.fit(path, dtype="pdf", chunk_size=10000, chunk_overlap=0)

In [4]:
llm = llms.GeminiModel()

In [5]:
fine_tuned_model = FineTuneEmbeddings()

In [6]:
embed_model = fine_tuned_model.train([path], "BAAI/bge-small-en-v1.5", llm, "fintune")

Parsing nodes: 100%|███████████████| 2/2 [00:00<00:00, 1025.13it/s]


Parsed 2 nodes


Parsing nodes: 100%|████████████████| 1/1 [00:00<00:00, 993.44it/s]


Parsed 1 nodes


100%|████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


...


Iteration: 100%|█████████████████████| 1/1 [00:00<00:00,  2.69it/s]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset after epoch 1:
Information Retrieval Evaluation of the model on the  dataset after epoch 1:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 4
Queries: 4
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 1

Corpus: 1

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cos_sim
Score-Function: cos_sim
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 100.00%
Accuracy@1: 100.00%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 100.00%
Accuracy@3: 100.00%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 100.00%
Accuracy@5: 100.00%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 100.00%
Accuracy@10: 100.00%
INFO:sentence_transformers.evaluation.Informatio


Epoch: 100%|█████████████████████████| 2/2 [00:02<00:00,  1.02s/it]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: fintune
Load pretrained SentenceTransformer: fintune


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [7]:
embed_model = fine_tuned_model.load_model("fintune")
retriever = retrieve.auto_retriever(data, embed_model, type="normal", top_k=4)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: fintune
Load pretrained SentenceTransformer: fintune
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [15]:
# print(retriever.retrieve("where was Alvin study at")[0].text)

In [9]:
pipeline = generator.Generate(question="Explain any alvin's cloud related work",retriever=retriever,llm=llm)

Batches: 100%|██████████████████████| 1/1 [00:00<00:00, 163.45it/s]


In [10]:
print(pipeline.call())

Alvin has experience with Google Cloud, Microsoft Azure, and FastAPI. He has used Google Cloud for Compute Engine, Cloud Run, Cloud Storage, Logging Sink, Pub/Sub, Cloud Build and Trigger, Cloud Artifact, Vertex, and more. He has also used Microsoft Azure and FastAPI for various projects.


In [11]:
print(pipeline.get_rag_triad_evals())

Executing RAG Triad Evaluations...
Context relevancy Score: 1.0
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.
Answer relevancy Score: 8.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
Groundness score: 7.0
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.
